In [3]:
import tensorflow as tf
import torch; from torch import tensor
import numpy as np

In [17]:
sigma:float = 0.1
min_sigma:float = 1e-3

A: tensor = tensor([[[0.2941, 0.8073, 0.5129, 0.7496, 0.3409, 0.6732],
         [0.9556, 0.2706, 0.5786, 0.9211, 0.4151, 0.5862],
         [0.3286, 0.5894, 0.1031, 0.2584, 0.5222, 0.4352]],

        [[0.0011, 0.8174, 0.6699, 0.6500, 0.9654, 0.7393],
         [0.4141, 0.9772, 0.8765, 0.0506, 0.5451, 0.1219],
         [0.9544, 0.2048, 0.0975, 0.1781, 0.4672, 0.7409]]]) # 2,3,6

B: tensor = tensor([[0.7245, 0.8128, 0.5149, 0.9719, 0.3836, 0.8181],
        [0.5655, 0.2984, 0.4779, 0.5923, 0.0746, 0.2779],
        [0.0882, 0.4241, 0.7142, 0.5485, 0.6926, 0.3199],
        [0.6485, 0.5530, 0.1981, 0.5301, 0.4493, 0.0664], 
        [0.6779, 0.8994, 0.1730, 0.4101, 0.5794, 0.1422]]) # 5,6

# Codigo Tensorflow

In [15]:
'''
Input:
    A: tensor of shape (bs, n, d)
    B: tensor of shape (m, d)
Result:
    K: tensor of shape (bs, n, m)
'''
shape_A = tf.shape(A)
shape_B = tf.shape(B)
# norma, luego colocarla en forma de columna y elevarla al cuardadro
A_norm = tf.norm(A, axis=-1)[..., tf.newaxis] ** 2 

B_norm = tf.norm(B, axis=-1)[tf.newaxis, tf.newaxis, :] ** 2
A_reshaped = tf.reshape(A, [-1, shape_A[2]])
AB = tf.matmul(A_reshaped, B, transpose_b=True) 
AB = tf.reshape(AB, [shape_A[0], shape_A[1], shape_B[0]])
dist2 = A_norm + B_norm - 2. * AB
dist2 = tf.clip_by_value(dist2, 0., np.inf)
sigma = tf.clip_by_value(sigma, min_sigma, np.inf) # TODO: Por qué realizar esto con tf
K = tf.exp(-dist2 / (2. * sigma ** 2.)) # type: ignore
K

<tf.Tensor: shape=(2, 3, 5), dtype=float32, numpy=
array([[[2.5594379e-06, 1.9049541e-13, 5.4484191e-12, 2.6279025e-16,
         1.7688930e-16],
        [1.3277054e-09, 3.3835313e-11, 3.3459910e-24, 7.3138919e-17,
         4.2679125e-25],
        [1.4964033e-23, 3.8320411e-14, 1.9762688e-13, 7.6061504e-08,
         5.2812929e-08]],

       [[2.3945422e-22, 3.2131640e-36, 5.9490668e-10, 4.1569465e-32,
         2.0833307e-28],
        [9.0981706e-36, 2.1843052e-26, 5.9128564e-17, 4.4353804e-21,
         5.8834750e-16],
        [1.1965393e-27, 4.5287348e-19, 2.9745520e-34, 3.4587660e-18,
         3.2644099e-22]]], dtype=float32)>

# From scrash in torch

In [27]:
res:tensor = A.unsqueeze(-2) - B
norma:tensor = torch.sum(res,dim=-1 )
valor_abs: tensor = torch.abs(norma)
division: tensor = valor_abs/ (2. *sigma**2.)
exponencial: tensor =  torch.exp(- division)
exponencial

tensor([[[3.8927e-19, 1.9978e-24, 1.5047e-13, 5.6085e-21, 1.6963e-11],
         [1.4895e-11, 5.2211e-32, 3.9326e-21, 1.4658e-28, 4.4332e-19],
         [6.4460e-44, 8.3325e-02, 1.1063e-12, 2.9681e-05, 9.8136e-15]],

        [[4.8952e-09, 1.5887e-34, 1.1966e-23, 4.4599e-31, 1.3489e-21],
         [1.1616e-27, 6.6950e-16, 5.0426e-05, 1.8795e-12, 5.6846e-03],
         [4.2439e-35, 1.8325e-08, 7.2452e-04, 5.1445e-05, 6.4270e-06]]])

In [43]:
tolerancia = 0.2

# Comparar los tensores con la tolerancia
similares = torch.allclose(tensor(K.numpy()), exponencial, atol=tolerancia)
similares

True